# Active Directory User Backdoors
Detects scenarios where one can control another users or computers account without having to use their credentials.

## Rule Content
```
- title: Active Directory User Backdoors
  id: 300bac00-e041-4ee2-9c36-e262656a6ecc
  description: Detects scenarios where one can control another users or computers
    account without having to use their credentials.
  references:
  - https://msdn.microsoft.com/en-us/library/cc220234.aspx
  - https://adsecurity.org/?p=3466
  - https://www.harmj0y.net/blog/redteaming/another-word-on-delegation/
  author: '@neu5ron'
  tags:
  - attack.t1098
  - attack.credential_access
  - attack.persistence
  logsource:
    product: windows
    service: security
    definition1: 'Requirements: Audit Policy : Account Management > Audit User Account
      Management, Group Policy : Computer Configuration\Windows Settings\Security
      Settings\Advanced Audit Policy Configuration\Audit Policies\Account Management\Audit
      User Account Management'
    definition2: 'Requirements: Audit Policy : DS Access > Audit Directory Service
      Changes, Group Policy : Computer Configuration\Windows Settings\Security Settings\Advanced
      Audit Policy Configuration\Audit Policies\DS Access\Audit Directory Service
      Changes'
    category: null
  detection:
    selection1:
      EventID: 4738
    filter1:
      AllowedToDelegateTo: null
    filter2:
      AllowedToDelegateTo: '-'
    selection2:
      EventID: 5136
      AttributeLDAPDisplayName: msDS-AllowedToDelegateTo
    selection3:
      EventID: 5136
      ObjectClass: user
      AttributeLDAPDisplayName: servicePrincipalName
    selection4:
      EventID: 5136
      AttributeLDAPDisplayName: msDS-AllowedToActOnBehalfOfOtherIdentity
    condition: (selection1 and not 1 of filter*) or selection2 or selection3 or selection4
  falsepositives:
  - Unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-security-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((((event_id:"4738" AND (NOT ((NOT _exists_:user_attribute_allowed_todelegate) OR (user_attribute_allowed_todelegate:"\-")))) OR (event_id:"5136" AND dsobject_attribute_name:"msDS\-AllowedToDelegateTo")) OR (event_id:"5136" AND dsobject_class:"user" AND dsobject_attribute_name:"servicePrincipalName")) OR (event_id:"5136" AND dsobject_attribute_name:"msDS\-AllowedToActOnBehalfOfOtherIdentity"))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()